In [1]:
using BenchmarkTools

In [2]:
_dimcheck_convolutionkernel(inputsize::Tuple{Int, Int}, kernelsize::Tuple{Int, Int}) = prod(inputsize .> kernelsize) || throw(ArgumentError("Kernel size $kernelsize cannot be larger than Input size $inputsize"))
_kernelsize(kernelsize::Tuple{Int, Int}, dilations::Tuple{Int, Int}) = ((dilations .* kernelsize) .- (dilations.-1))
_featuremapsize(inputsize::Tuple{Int, Int}, kernelsize::Tuple{Int, Int}, strides::Tuple{Int, Int}) = div.(inputsize .- kernelsize, strides) .+ 1
_convolutionkernelmargin(inputsize::Tuple{Int, Int}, kernelsize::Tuple{Int, Int}) = Int.(ceil.(inputsize .% kernelsize))

_convolutionkernelmargin (generic function with 1 method)

In [3]:
i = (28,28,3,1)
k = (3,3,6)
s = (1,1)
d = (1,1)

(1, 1)

In [4]:
input_size = i[1:end-2]
kernel_size = _kernelsize(k[1:end-1], d)
fm_size = _featuremapsize(input_size, kernel_size, s)
ws = (k[1:2]...,i[3],k[3])
bs = (1,1,k[3],1)
os = (fm_size...,k[end], i[end])

(26, 26, 6, 1)

In [5]:
w = rand(ws...)

3×3×3×6 Array{Float64,4}:
[:, :, 1, 1] =
 0.85376   0.665462  0.783893
 0.733304  0.176091  0.24583 
 0.350374  0.605851  0.463537

[:, :, 2, 1] =
 0.263124   0.63709   0.959691
 0.772458   0.870125  0.386252
 0.0862311  0.989644  0.506057

[:, :, 3, 1] =
 0.196031    0.0932571  0.153073
 0.723194    0.769432   0.84717 
 0.00365026  0.543455   0.24336 

[:, :, 1, 2] =
 0.738066  0.303038  0.811583
 0.943426  0.797127  0.479646
 0.886674  0.309054  0.177252

[:, :, 2, 2] =
 0.93017   0.04378   0.019232
 0.744943  0.540822  0.131536
 0.452981  0.203196  0.20208 

[:, :, 3, 2] =
 0.26635   0.0216547  0.318762
 0.188547  0.910467   0.426798
 0.646105  0.591617   0.44496 

[:, :, 1, 3] =
 0.10383   0.592006   0.363534
 0.175676  0.0164652  0.21828 
 0.864373  0.284846   0.30568 

[:, :, 2, 3] =
 0.709468  0.282646  0.329642
 0.365907  0.195063  0.560977
 0.580107  0.398533  0.128019

[:, :, 3, 3] =
 0.489444  0.586143  0.0019539
 0.818326  0.379876  0.68677  
 0.572898  0.653146  0.501435 


In [6]:
"Creates convolution matrix according to given input, kernel, stride and dilation configuration"
function im2col(conv_matrix, conv_kernel, input_size::Tuple{Int, Int}, kernel_size::Tuple{Int, Int}, strides::Tuple{Int, Int}, dilations::Tuple{Int, Int}, fm_size::Tuple{Int, Int})
    margin = input_size.%fm_size
    conv_matrix.=0
    t1 = zeros(input_size)
    for ci in 1:size(conv_kernel)[4]
        for ki in 1:size(conv_kernel)[3]
            idx = 1
            for i in 1:strides[1]:(fm_size[1]*strides[1])
                for j in 1:strides[2]:(fm_size[2]*strides[2])
                    t1.=0
                    t1[i:dilations[1]:(i+kernel_size[1]-1), j:dilations[2]:(j+kernel_size[2]-1)] = conv_kernel[:,:,ki,ci]'
                    conv_matrix[idx,:,ki,ci] .= t1'[:]
                    idx+=1
                end
            end
        end
    end
end

im2col

In [7]:
conv_mat = zeros(prod(fm_size), prod(input_size), size(w)[3:end]...)

676×784×3×6 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [8]:
im2col(conv_mat, w, input_size, kernel_size, s, d, fm_size)

In [9]:
conv_mat

676×784×3×6 Array{Float64,4}:
[:, :, 1, 1] =
 0.85376  0.733304  0.350374  0.0       …  0.0       0.0       0.0     
 0.0      0.85376   0.733304  0.350374     0.0       0.0       0.0     
 0.0      0.0       0.85376   0.733304     0.0       0.0       0.0     
 0.0      0.0       0.0       0.85376      0.0       0.0       0.0     
 0.0      0.0       0.0       0.0          0.0       0.0       0.0     
 0.0      0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0      0.0       0.0       0.0          0.0       0.0       0.0     
 0.0      0.0       0.0       0.0          0.0       0.0       0.0     
 0.0      0.0       0.0       0.0          0.0       0.0       0.0     
 0.0      0.0       0.0       0.0          0.0       0.0       0.0     
 0.0      0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0      0.0       0.0       0.0          0.0       0.0       0.0     
 0.0      0.0       0.0       0.0          0.0       0.0       0.0     
 ⋮                 

In [10]:
"Fetches convolution kernel from given convolution matrix, input_size, kernel size, stride and dilation configuration"
function col2im(conv_matrix, conv_kernel, input_size::Tuple{Int, Int}, weight_size::Tuple{Int, Int}, strides::Tuple{Int, Int}, dilations::Tuple{Int, Int}, fm_size::Tuple{Int, Int})
    kernel_size = size(conv_kernel)
    margin = _convolutionkernelmargin(input_size, kernel_size[1:2])#

    conv_kernel .= 0
    for ci in 1:kernel_size[4]
        for ki in 1:kernel_size[3]
            cki = 1
            for i in (1+margin[1]):strides[1]:(input_size[1]-margin[2])
                for j in (1+margin[1]):strides[2]:(input_size[2]-margin[2])
                    temp_ck = reshape(conv_matrix[cki,:,ki,ci], input_size)'[i-margin[1]:dilations[1]:(i+(margin[1])), j-margin[1]:dilations[2]:(j+(margin[1]))]
                    conv_kernel[:,:,ki,ci].+=temp_ck'
                    cki+=1
                end
            end
            conv_kernel[:,:,ki,ci]./=(size(conv_matrix)[1])
        end
    end
end

col2im

In [11]:
col2im(conv_mat, w, input_size, ws[1:2], s, d, fm_size)

In [12]:
w

3×3×3×6 Array{Float64,4}:
[:, :, 1, 1] =
 0.85376   0.665462  0.783893
 0.733304  0.176091  0.24583 
 0.350374  0.605851  0.463537

[:, :, 2, 1] =
 0.263124   0.63709   0.959691
 0.772458   0.870125  0.386252
 0.0862311  0.989644  0.506057

[:, :, 3, 1] =
 0.196031    0.0932571  0.153073
 0.723194    0.769432   0.84717 
 0.00365026  0.543455   0.24336 

[:, :, 1, 2] =
 0.738066  0.303038  0.811583
 0.943426  0.797127  0.479646
 0.886674  0.309054  0.177252

[:, :, 2, 2] =
 0.93017   0.04378   0.019232
 0.744943  0.540822  0.131536
 0.452981  0.203196  0.20208 

[:, :, 3, 2] =
 0.26635   0.0216547  0.318762
 0.188547  0.910467   0.426798
 0.646105  0.591617   0.44496 

[:, :, 1, 3] =
 0.10383   0.592006   0.363534
 0.175676  0.0164652  0.21828 
 0.864373  0.284846   0.30568 

[:, :, 2, 3] =
 0.709468  0.282646  0.329642
 0.365907  0.195063  0.560977
 0.580107  0.398533  0.128019

[:, :, 3, 3] =
 0.489444  0.586143  0.0019539
 0.818326  0.379876  0.68677  
 0.572898  0.653146  0.501435 


In [13]:
function conv(w, x;s=(1,1),d=(1,1))
    x_size = size(x)
    x_channels = x_size[3]
    x_batch = x_size[4]

    kernel_size = _kernelsize(size(w)[1:2], d)
    fm_size = _featuremapsize(x_size[1:2], kernel_size, s)

    cm_size = (prod(fm_size), prod(x_size[1:2]), size(w)[3:end]...)
    cm_channels = cm_size[4]    
    conv_mat=zeros(cm_size...)
    
    reshaped_x = reshape(x, (prod(size(x)[1:3]), x_batch))
    
    im2col(conv_mat, w, x_size[1:2], kernel_size, s, d, fm_size)
    
    return reshape(cat([reshape(reshape(conv_mat[:,:,:,cmi], (cm_size[1], prod(cm_size[2:3])))*reshaped_x, (prod(fm_size), 1, x_batch)) for cmi in 1:cm_channels]...,dims=2), (fm_size..., cm_channels, x_batch))
end

conv (generic function with 1 method)

In [14]:
inp = rand(28,28,3,1)

28×28×3×1 Array{Float64,4}:
[:, :, 1, 1] =
 0.917239   0.400986   0.534921   …  0.370253   0.0519467  0.419958 
 0.224137   0.320092   0.914086      0.84912    0.264695   0.0089115
 0.834904   0.394403   0.430203      0.209478   0.640072   0.474872 
 0.494023   0.164034   0.171776      0.35664    0.975366   0.155729 
 0.073049   0.698742   0.404488      0.862001   0.432985   0.548096 
 0.464037   0.438487   0.749432   …  0.890446   0.292835   0.274033 
 0.750998   0.646687   0.240642      0.571676   0.874348   0.766436 
 0.0117564  0.986178   0.657663      0.753055   0.334359   0.585372 
 0.216284   0.449581   0.324258      0.914127   0.137015   0.262491 
 0.356625   0.0583815  0.390379      0.101644   0.0595431  0.974367 
 0.108282   0.0811096  0.26739    …  0.549431   0.239904   0.192431 
 0.0796118  0.920028   0.135287      0.17359    0.426027   0.781982 
 0.43295    0.852961   0.144844      0.101065   0.666477   0.160467 
 ⋮                                ⋱  ⋮                      

In [15]:
out = conv(w, inp)

26×26×6×1 Array{Float64,4}:
[:, :, 1, 1] =
 7.32075  6.03455  6.461    6.30533  …  7.23897  5.32927  5.82763  6.66985
 7.21353  6.92629  6.44398  7.13198     7.2493   7.34409  7.36142  7.02206
 7.80277  7.23802  6.38284  5.79533     7.57613  7.5079   8.34785  6.87905
 8.54299  7.60998  6.84291  7.25054     5.69851  6.63876  8.29268  7.59036
 7.58982  7.64765  6.18172  6.80583     6.84691  7.53973  8.33312  7.58328
 7.14392  7.80725  5.98608  6.43716  …  5.34978  6.61535  7.6598   6.34234
 6.65868  6.82638  6.411    6.4217      5.97287  7.57663  7.4914   7.69881
 7.40657  7.47378  5.95044  6.93        6.82855  7.24817  5.78769  7.05788
 5.89131  7.15726  6.93077  6.5445      6.61256  5.73499  5.3198   6.10189
 6.41891  6.32202  5.35018  5.71259     6.78736  6.27664  5.09545  5.61352
 6.30215  6.67914  5.21022  6.38252  …  7.86456  7.10739  5.70988  5.84951
 7.81215  7.85337  5.79257  7.24083     7.64993  7.05269  5.99248  5.70123
 7.17887  7.33643  5.99606  8.09194     7.27506  7.00685 

In [32]:
#676,784 * 784,1 -> 676,1
#676,1T===1,676 * 676, 784 -> 1, 784
function convx(w, x, dy;s=(1,1),d=(1,1))
    x_size = size(x)
    x_channels = x_size[3]
    x_batch = x_size[4]

    kernel_size = _kernelsize(size(w)[1:2], d)
    fm_size = _featuremapsize(x_size[1:2], kernel_size, s)

    cm_size = (prod(fm_size), prod(x_size[1:2]), size(w)[3:end]...)
    cm_channels = cm_size[4]    
    conv_mat=zeros(cm_size...)
    
    im2col(conv_mat, w, x_size[1:2], kernel_size, s, d, fm_size)
    conv_mat = permutedims(conv_mat, (2,1,3,4))
    conv_mat = permutedims(conv_mat, (1,2,4,3))
    cm_size = size(conv_mat)
    cm_channels = cm_size[4]    

    dy_size = size(dy)
    dy_batch=dy_size[4]
    reshaped_dy = reshape(dy, (prod(dy_size[1:3]), dy_batch))

    return reshape(cat([reshape(reshape(conv_mat[:,:,:,cmi], (cm_size[1], prod(cm_size[2:3])))*reshaped_dy, (prod(x_size[1:2]), 1, x_batch)) for cmi in 1:cm_channels]...,dims=2), (x_size[1:2]..., cm_channels, x_batch))
end

convx (generic function with 1 method)

In [33]:
convx(w, inp, out)

(784, 676, 6, 3)
(4056, 1)


28×28×3×1 Array{Float64,4}:
[:, :, 1, 1] =
 15.2471   38.6295   62.2616   60.1824  …   57.697    42.5881  21.857  
 43.9694   82.1486  119.647   119.182      117.863    75.2373  35.9852 
 67.7466  119.41    165.27    159.344      165.28    102.431   46.8412 
 70.9491  126.554   172.38    159.691      175.066   108.059   49.768  
 73.7913  130.953   174.553   166.107      181.026   107.457   50.2687 
 73.7324  130.766   172.612   164.859   …  175.782   104.707   48.7557 
 67.9454  123.088   158.111   155.16       172.607   105.637   51.0029 
 62.6325  119.804   158.517   154.134      166.341    98.6826  49.31   
 60.2361  113.809   152.952   155.97       158.049    90.016   45.6562 
 58.0474  111.139   151.582   150.503      142.378    82.1109  41.7148 
 55.6877  107.793   148.24    149.727   …  143.229    81.0872  38.2993 
 60.7974  120.484   155.936   155.823      146.343    84.0574  39.6426 
 66.4682  124.065   156.839   161.935      153.764    91.8851  42.6946 
  ⋮                  

In [34]:
function convw(w, x, dy;s=(1,1),d=(1,1))
    x_size = size(x)
    x_channels = x_size[3]
    x_batch = x_size[4]

    kernel_size = _kernelsize(size(w)[1:2], d)
    fm_size = _featuremapsize(x_size[1:2], kernel_size, s)

    cm_size = (prod(fm_size), prod(x_size[1:2]), size(w)[3:end]...)
    cm_channels = cm_size[4]    
    conv_mat=zeros(cm_size...)
    
    dy = permutedims(dy, (1,2,4,3))
    dy_size = size(dy)
    dy_batch=dy_size[3]
    dy_channels=dy_size[4]
    reshaped_dy = reshape(dy, (prod(size(dy)[1:2]), dy_batch, dy_channels))
    
    reshaped_x = reshape(x, (prod(size(x)[1:3]), x_batch))'
    im2col(conv_mat, w, x_size[1:2], kernel_size, s, d, fm_size)

    dw = cat([reshape(reshape(reshaped_dy[:,:,dmi], (prod(dy_size[1:2]), dy_batch))*reshaped_x, (prod(dy_size[1:2]), prod(x_size[1:2]), x_channels, 1)) for dmi in 1:dy_channels]...,dims=4)
    
    return col2im(dw, w, x_size[1:2], size(w)[1:2], s, d, fm_size)
end

convw (generic function with 1 method)

In [35]:
convw(w, inp, out)

In [36]:
w

3×3×3×6 Array{Float64,4}:
[:, :, 1, 1] =
 3.48723  3.49652  3.49463
 3.48692  3.47719  3.47667
 3.44914  3.5165   3.49002

[:, :, 2, 1] =
 3.56105  3.59768  3.61241
 3.57219  3.56622  3.50639
 3.5269   3.59424  3.5412 

[:, :, 3, 1] =
 3.49327  3.50546  3.52447
 3.56984  3.59062  3.61801
 3.47463  3.52726  3.50199

[:, :, 1, 2] =
 3.13136  3.1095   3.14048
 3.15846  3.17244  3.14784
 3.14497  3.1276   3.12255

[:, :, 2, 2] =
 3.26084  3.18176  3.17126
 3.21732  3.18093  3.12907
 3.19617  3.16341  3.14849

[:, :, 3, 2] =
 3.14995  3.14551  3.17802
 3.1644   3.24609  3.21584
 3.17623  3.17221  3.17226

[:, :, 1, 3] =
 2.75603  2.80277  2.77498
 2.77063  2.77478  2.8017 
 2.81889  2.80177  2.79021

[:, :, 2, 3] =
 2.89899  2.86231  2.85454
 2.84887  2.8273   2.83193
 2.872    2.86056  2.81207

[:, :, 3, 3] =
 2.82919  2.85566  2.80899
 2.88102  2.86745  2.89921
 2.83647  2.84383  2.84637

[:, :, 1, 4] =
 2.96092  2.94325  2.96529
 3.01232  2.97211  2.97554
 2.94806  2.97934  2.95065

[:, 